In [1]:
import os
import sys
import socket
import numpy as np
import pandas as pd #also requires xlsxwriter
from at_synapse_detection import dataAccess as da
from at_synapse_detection import antibodyAnalysis as aa

In [2]:
"""Compare two PSD-95 clones.  This is just one way of setting up the 
queries/data locations.  See the 'antibody_analysis' folder for multiple examples 
of how to set up the tool. 

Query Format (dict): 
    preIF : list of strs - name of the channel
    preIF_z : list of ints - number of slices to span. default=[2]
    postIF : list of strs
    postIF_z : list of ints. default=[2]
    punctumSize : number of pixels x/y that a puncta should span. default=2

Returns
-----------------
df : dataframe - contains the results of the SACT 
"""

"Compare two PSD-95 clones.  This is just one way of setting up the \nqueries/data locations.  See the 'antibody_analysis' folder for multiple examples \nof how to set up the tool. \n\nQuery Format (dict): \n    preIF : list of strs - name of the channel\n    preIF_z : list of ints - number of slices to span. default=[2]\n    postIF : list of strs\n    postIF_z : list of ints. default=[2]\n    punctumSize : number of pixels x/y that a puncta should span. default=2\n\nReturns\n-----------------\ndf : dataframe - contains the results of the SACT \n"

In [3]:
# Location of the data 
base_dir = '../example/' 

In [4]:
# Data resolution, in nanometers 
resolution = {'res_xy_nm': 100, 'res_z_nm': 70}

In [5]:
# Threshold value for the probability maps. This value does not usually need to be changed. 
thresh = 0.9

In [9]:
# List the file names.  These must be the same length
presynaptic_channels = ['synapsin_1st.tif']
presynaptic_z_list = [2]

postsynaptic_channels = ['PSD95m_1st.tif', 'PSD95r_2nd.tif']
postsynaptic_z_list = [2, 2]



In [10]:
target_filenames = postsynaptic_channels
target_number_of_slices = postsynaptic_z_list
reference_filenames = presynaptic_channels
reference_number_of_slices = presynaptic_z_list

In [14]:
# Create a query for each pair
query_list = []

target_name = postsynaptic_channels[0] # The AB we're interested in testing (PSD)
reference_name = presynaptic_channels[0] # The previously validated AB (synapsin)

# Formulate the query
query = {'preIF': presynaptic_channels, 'preIF_z': presynaptic_z_list, 
         'postIF': postsynaptic_channels, 'postIF_z': postsynaptic_z_list,
         'punctumSize': 2}
query_list.append(query)

# Run the SACT 
measure_list = aa.calculate_measure_lists(query_list, None, base_dir,
                                    thresh, resolution, target_filenames)

# Convert measure object to a dataframe 
project_names = ['Data1']
df = aa.create_df(measure_list, project_names, target_filenames, reference_filenames)
print(df)
df_list = [df]

synapsin_1st.tif
PSD95m_1st.tif
PSD95r_2nd.tif
data volume:  10104.640000000001 um3
Computed presynaptic single channel measurements
Computed postsynaptic single channel measurements
starting z ind: 0
starting z ind: 1
starting z ind: 2
starting z ind: 3
starting z ind: 4
starting z ind: 5
starting z ind: 6
starting z ind: 7
starting z ind: 8
starting z ind: 9
            Target AB      Conjugate AB  Puncta Density  Puncta Volume  \
Data1  PSD95m_1st.tif  synapsin_1st.tif         0.94798      10.663535   

       Puncta STD  Synapse Density       TSR  Raw Mean    Raw STD  \
Data1  101.691574          0.21307  0.224763  3.004389  11.715906   

       Signal Mean  Signal STD  
Data1    44.793766   32.018479  


In [12]:
#Output filename
fn = 'sact_psd_example.xlsx' 
sheet_name = 'sact_psd_example'

In [13]:
# Determine if file exists. This is only an issue when running on windows
if  os.path.isfile(fn): 
    print('A sheet with this name already exists; rename sheet and rerun')
    #sys.exit()

# Export the dataframe to excel 
aa.write_dfs_to_excel(df_list, sheet_name, fn)

A sheet with this name already exists; rename sheet and rerun
